# XML Generator for CrossRef

This notebook generates the XML file for CrossRef using the output from the DL Batch Revise Export.

For this one we're following the example metadata for Reports: [https://gitlab.com/crossref/schema/-/blob/master/best-practice-examples/report.5.3.0.xml](https://gitlab.com/crossref/schema/-/blob/master/best-practice-examples/report.5.3.0.xml)


In [ ]:
import pandas as pd
from lxml import etree
from datetime import datetime

## VARIABLES

These are the variables you have to modify before you run the notebook.

In [ ]:
# VARIABLES AND PARAMETERS YOU HAVE TO SET EVERY RUN.
today = datetime.now()

SUBMISSION_TIMESTAMP = today.strftime("%Y%m%d%H%M%S") + "0000"

CSV_IN_FILE_NAME = "book_reviews"

## Generate the Root Element

In [ ]:
# Create a function to check the state of XML at any point:

def view_xml_result():
    # Pretty-print the XML
    xml_string = etree.tostring(doi_batch, pretty_print=True, xml_declaration=True, encoding="utf-8").decode("utf-8")

    # Print the formatted XML
    print(xml_string)

In [ ]:
# Define namespaces without 'xmlns:' in the keys
namespaces = {
    "xsi": "http://www.w3.org/2001/XMLSchema-instance",
    None: "http://www.crossref.org/schema/5.3.0",  # Default namespace
    "jats": "http://www.ncbi.nlm.nih.gov/JATS1",
    "fr": "http://www.crossref.org/fundref.xsd",
    "mml": "http://www.w3.org/1998/Math/MathML",
}

# Create the root element with namespaces
doi_batch = etree.Element("doi_batch", nsmap=namespaces)

doi_batch.set("version", "5.3.0")

doi_batch.set(
    "{http://www.w3.org/2001/XMLSchema-instance}schemaLocation",
    "http://www.crossref.org/schema/5.3.0 https://www.crossref.org/schemas/crossref5.3.0.xsd"
)

# Add head and body
head = etree.SubElement(doi_batch, "head")
body = etree.SubElement(doi_batch, "body")

view_xml_result()

<?xml version='1.0' encoding='utf-8'?>
<doi_batch xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.crossref.org/schema/5.3.0" xmlns:jats="http://www.ncbi.nlm.nih.gov/JATS1" xmlns:fr="http://www.crossref.org/fundref.xsd" xmlns:mml="http://www.w3.org/1998/Math/MathML" version="5.3.0" xsi:schemaLocation="http://www.crossref.org/schema/5.3.0 https://www.crossref.org/schemas/crossref5.3.0.xsd">
  <head/>
  <body/>
</doi_batch>



## Populate the `head`

In [ ]:
today_short = today.strftime("%y%m%d")

doi_batch_id = etree.SubElement(head, "doi_batch_id").text = f"reviews.{today_short}"
timestamp = etree.SubElement(head, "timestamp").text = SUBMISSION_TIMESTAMP

depositor = etree.SubElement(head, "depositor")
depositor_name = etree.SubElement(depositor, "depositor_name").text = "desres:desres"
email_address = etree.SubElement(depositor, "email_address").text = ("dl@designresearchsociety.org")

registrant = etree.SubElement(head, "registrant").text = "Digital Library"

view_xml_result()

<?xml version='1.0' encoding='utf-8'?>
<doi_batch xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.crossref.org/schema/5.3.0" xmlns:jats="http://www.ncbi.nlm.nih.gov/JATS1" xmlns:fr="http://www.crossref.org/fundref.xsd" xmlns:mml="http://www.w3.org/1998/Math/MathML" version="5.3.0" xsi:schemaLocation="http://www.crossref.org/schema/5.3.0 https://www.crossref.org/schemas/crossref5.3.0.xsd">
  <head>
    <doi_batch_id>reviews.250304</doi_batch_id>
    <timestamp>202503041838100000</timestamp>
    <depositor>
      <depositor_name>desres:desres</depositor_name>
      <email_address>dl@designresearchsociety.org</email_address>
    </depositor>
    <registrant>Digital Library</registrant>
  </head>
  <body/>
</doi_batch>



# Populating the `body`

Body has a series of `report-paper` elements. Each element has the following structure:

- `contributors`
    - `person_name`
        - `given_name`
        - `surname`
        - `affiliations`
            - `institution`
                - `institution_name`
- `titles`
    - `title`
- `edition_number` == 0
- `publication_date`
    - `month`
    - `day`
    - `year`
- `doi_data`
    - `doi`
    - `resource`


We'll try to respect this and see if we have to add more info for it to pass.

---

## Creating Report Papers.


First we load the CSV file and remove the unneeded columns. For our purposes we need the following columns:

- `author1_fname`
- `author1_mname`
- `author1_lname`
- `author1_suffix`
- `author1_institution`
- `title`
- `publication_date`
- `doi`
- `calc_url`

In [ ]:
# LOAD CSV FILE
df = pd.read_csv(CSV_IN_FILE_NAME+".csv", sep=",")


df_core = df[["title", "publication_date", "doi", "calc_url", "author1_fname", "author1_mname", "author1_lname", "author1_suffix", "author1_institution"]]
df_core = df_core.fillna('').astype(str)
df_core.head(10)

,title,publication_date,doi,calc_url,author1_fname,author1_mname,author1_lname,author1_suffix,author1_institution
0,Review of The Legal Design Book: Doing Law in ...,2022-03-28 00:00,10.21606/wsbr.001,https://dl.designresearchsociety.org/drs-book-...,Mohammed,,Al Rezan,,Al-Imam Mohamed Ibn Saud Islamic University
1,"Review of Design, Empathy, Interpretation: Tow...",2023-10-27 00:00,10.21606/wsbr.002,https://dl.designresearchsociety.org/drs-book-...,Jules,Rochielle,Sievert,,Northeastern University
2,Review of Design for a Better World: Meaningfu...,2023-11-08 00:00,10.21606/wsbr.003,https://dl.designresearchsociety.org/drs-book-...,Shriyash,,Shete,,
3,Review of Design is Power: The Dark Side,2025-01-01 00:00,10.21606/wsbr.004,https://dl.designresearchsociety.org/drs-book-...,Samantha,,Osys,,The Open University
4,"Review of Design Social Change: take action, w...",2024-02-01 00:00,10.21606/wsbr.005,https://dl.designresearchsociety.org/drs-book-...,Michael,,Kibedi,,
5,Review of Designing for Sex and Gender Equity,2024-04-01 00:00,10.21606/wsbr.006,https://dl.designresearchsociety.org/drs-book-...,Annapaola,,Vacanti,,Università Iuav di Venezia
6,Review of Bioprotopia: Designing the Built Env...,2024-05-01 00:00,10.21606/wsbr.007,https://dl.designresearchsociety.org/drs-book-...,Gozde,Damla,Turhan-Haskara,,Izmir University of Economics
7,Review of Meaningful Stuff. Design That Lasts.,2024-06-01 00:00,10.21606/wsbr.008,https://dl.designresearchsociety.org/drs-book-...,Niklas,Hermann,Henke,,University of Grenoble
8,"Review of Teaching Graphic Design: Approaches,...",2024-06-01 00:00,10.21606/wsbr.009,https://dl.designresearchsociety.org/drs-book-...,Ellen,,Christensen,,San Francisco State University
9,Review of Visual Methods for Digital Research:...,2024-10-01 00:00,10.21606/wsbr.010,https://dl.designresearchsociety.org/drs-book-...,Gerry,,Derksen,,Clemson University


Now we use this data to create each `report-paper` element in the XML.

In [ ]:
# Clean body so we can try again
body.clear()

# Create element for each row in the table
for index, row in df_core.iterrows():
    print(f"Working on row {index}: {row['title']}")


    # Create <report-paper> and <report-paper_metadata>.
    report_paper = etree.SubElement(body, "report-paper")
    report_paper_metadata = etree.SubElement(report_paper, "report-paper_metadata",language="en")

    # Create contributor
    contributor = etree.SubElement(report_paper_metadata, "contributors")

    # contributor - Add person name
    person_name = etree.SubElement(contributor, "person_name", sequence="first", contributor_role="author")
    given_name = etree.SubElement(person_name, "given_name").text = row['author1_fname']
    surname = etree.SubElement(person_name, "surname").text = row['author1_lname']

    # contributor - Add affiliation
    affiliations = etree.SubElement(person_name, "affiliations")
    institution = etree.SubElement(affiliations, "institution")
    institution_name = etree.SubElement(institution, "institution_name").text = (row['author1_institution'].strip() or "Design Research Society")

    # Create titles
    titles = etree.SubElement(report_paper_metadata, "titles")
    title = etree.SubElement(titles, "title").text = row['title']

    # Create edition_number
    edition_number = etree.SubElement(report_paper_metadata, "edition_number").text = "0"

    # Create publication_date
    publication_date = etree.SubElement(report_paper_metadata, "publication_date")

    # publication_date - clean date
    date_part = row['publication_date'].split(' ')[0]
    year_data, month_data, day_data = date_part.split('-')

    # publication_date - add elements
    month = etree.SubElement(publication_date, "month").text = month_data
    day = etree.SubElement(publication_date, "day").text = day_data
    year = etree.SubElement(publication_date, "year").text = year_data


    # Create doi_data
    doi_data = etree.SubElement(report_paper_metadata, "doi_data")

    # doi_data - add elements
    doi = etree.SubElement(doi_data, "doi").text = row["doi"].replace("doi.org/", "")
    resource = etree.SubElement(doi_data, "resource").text = row["calc_url"]


print("\n\n\n")
view_xml_result()

Working on row 0: Review of The Legal Design Book: Doing Law in the 21st Century
Working on row 1: Review of Design, Empathy, Interpretation: Toward Interpretive Design Research
Working on row 2: Review of Design for a Better World: Meaningful, Sustainable, Humanity Centered
Working on row 3: Review of Design is Power: The Dark Side
Working on row 4: Review of Design Social Change: take action, work toward equity, and challenge the status quo
Working on row 5: Review of Designing for Sex and Gender Equity
Working on row 6: Review of Bioprotopia: Designing the Built Environment with Living Organisms
Working on row 7: Review of Meaningful Stuff. Design That Lasts.
Working on row 8: Review of Teaching Graphic Design: Approaches, Insights, the Role of Listening, and 24 Interviews with Inspirational Educators
Working on row 9: Review of Visual Methods for Digital Research: An Introduction
Working on row 10: Review of Design Thinking and Other Approaches—How Different Disciplines See, Think 

SAVE THE XML

In [ ]:
xml_string = etree.tostring(doi_batch, pretty_print=True, xml_declaration=True, encoding="utf-8").decode("utf-8")

output_file = f"{CSV_IN_FILE_NAME}.xml"
# Open the file in write mode and save the XML string
with open(output_file, "w", encoding="utf-8") as f:
    f.write(xml_string)

print(f"XML has been saved to {output_file}")

XML has been saved to book_reviews.xml
